In [5]:
%pip install pyrosm geopandas folium

In [6]:
import warnings
warnings.filterwarnings(action="ignore")
import requests
import time
import geopandas as gpd
import tempfile
import os
from io import StringIO

In [ ]:
BIKE_DATASET_ID = "d_9326f791b521187f503149712fc400ef"
ROAD_DATASET_ID = "d_95a29fbb10cf94a3c263d33861d7b6c6"
ACRA_DATASET_ID='d_3f960c10fed6145404ca7b821f263b87'

In [7]:
def load_geojson_as_gdf(DATASET_ID, max_tries=3):
    INITIATE_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download"
    POLL_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download"

    init_resp = requests.get(INITIATE_URL)
    init_resp.raise_for_status()

    for _ in range(max_tries):
        time.sleep(2)
        poll_resp = requests.get(POLL_URL)
        poll_resp.raise_for_status()
        download_url = poll_resp.json().get("data", {}).get("url")
        if download_url:
            break
    else:
        raise TimeoutError("Timed out waiting for dataset download URL.")

    with tempfile.NamedTemporaryFile(delete=False, suffix=".geojson") as tmpfile:
        resp = requests.get(download_url)
        resp.raise_for_status()
        tmpfile.write(resp.content)
        tmpfile.flush()
        tmpfile.close()
        gdf = gpd.read_file(tmpfile.name)
    os.remove(tmpfile.name)
    return gdf

In [ ]:
bike_gdf = load_geojson_as_gdf(BIKE_DATASET_ID)
road_gdf=load_geojson_as_gdf(ROAD_DATASET_ID)

In [8]:
road_gdf.head()

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83972 1.35139 0, 103.83972 1..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83989 1.35113 0, 103.83989 1..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.82322 1.40792 0, 103.82301 1..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.82283 1.4119 0, 103.8228 1.4..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.82256 1.41115 0, 103.82253 1..."


In [11]:
import requests
import pandas as pd
import zipfile
import io

headers = {
    "AccountKey": "rkRkeQReQ/CPdWz43lZtRA==",
    "accept": "application/json"
}

params = {"Date": "202506"}
url = "https://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain"
resp = requests.get(url, headers=headers, params=params)
resp.raise_for_status()
download_link = resp.json()["value"][0]["Link"]

zip_resp = requests.get(download_link)
zip_file = zipfile.ZipFile(io.BytesIO(zip_resp.content))

csv_name = [name for name in zip_file.namelist() if name.endswith(".csv")][0]
with zip_file.open(csv_name) as csvfile:
    od_train_df = pd.read_csv(csvfile)


In [12]:
od_train_df.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2025-06,WEEKDAY,17,TRAIN,NS9/TE2,DT29,43
1,2025-06,WEEKDAY,22,TRAIN,NE11,EW25,5
2,2025-06,WEEKDAY,16,TRAIN,DT29,SW6,3
3,2025-06,WEEKDAY,19,TRAIN,NS27/CE2/TE20,CC12,102
4,2025-06,WEEKENDS/HOLIDAY,11,TRAIN,EW7,PW4,1


In [13]:
import requests
import pandas as pd
import zipfile
import io

headers = {
    "AccountKey": "rkRkeQReQ/CPdWz43lZtRA==",
    "accept": "application/json"
}

params = {"Date": "202506"}
url = "https://datamall2.mytransport.sg/ltaodataservice/PV/ODBus"
resp = requests.get(url, headers=headers, params=params)
resp.raise_for_status()
download_link = resp.json()["value"][0]["Link"]

zip_resp = requests.get(download_link)
zip_file = zipfile.ZipFile(io.BytesIO(zip_resp.content))

csv_name = [name for name in zip_file.namelist() if name.endswith(".csv")][0]
with zip_file.open(csv_name) as csvfile:
    od_bus_df = pd.read_csv(csvfile)

print(od_bus_df.head())


  YEAR_MONTH DAY_TYPE  TIME_PER_HOUR PT_TYPE  ORIGIN_PT_CODE  \
0    2025-06  WEEKDAY             15     BUS           83101   
1    2025-06  WEEKDAY             17     BUS           98139   
2    2025-06  WEEKDAY             18     BUS           66359   
3    2025-06  WEEKDAY              6     BUS            7221   
4    2025-06  WEEKDAY              8     BUS           19011   

   DESTINATION_PT_CODE  TOTAL_TRIPS  
0                71159            6  
1                63219           30  
2                62141           39  
3                60121           52  
4                28009            5  


In [14]:
od_bus_df

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2025-06,WEEKDAY,15,BUS,83101,71159,6
1,2025-06,WEEKDAY,17,BUS,98139,63219,30
2,2025-06,WEEKDAY,18,BUS,66359,62141,39
3,2025-06,WEEKDAY,6,BUS,7221,60121,52
4,2025-06,WEEKDAY,8,BUS,19011,28009,5
...,...,...,...,...,...,...,...
5609764,2025-06,WEEKDAY,12,BUS,82099,92151,7
5609765,2025-06,WEEKENDS/HOLIDAY,9,BUS,76259,76099,1
5609766,2025-06,WEEKENDS/HOLIDAY,22,BUS,68019,53229,2
5609767,2025-06,WEEKENDS/HOLIDAY,15,BUS,12119,4179,1


In [15]:
def load_dataset_as_dataframe(DATASET_ID,max_tries=3):
    INITIATE_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download"
    POLL_URL = f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download"
    init_resp = requests.get(INITIATE_URL)
    init_resp.raise_for_status()

    for _ in range(max_tries):
        time.sleep(2)
        poll_resp = requests.get(POLL_URL)
        poll_resp.raise_for_status()
        download_url = poll_resp.json().get("data", {}).get("url")
        if download_url:
            break
    else:
        raise TimeoutError("Timed out waiting for dataset download URL.")

    csv_resp = requests.get(download_url)
    csv_resp.raise_for_status()
    df = pd.read_csv(StringIO(csv_resp.text))
    return df

In [36]:
acra_df = load_dataset_as_dataframe(ACRA_DATASET_ID)

In [37]:
len(acra_df['reg_postal_code'].unique())

81746

In [20]:
import gzip
import json
import pandas as pd
import requests

url = "https://github.com/isen-ng/singapore-postal-codes-1/raw/master/singpostcode.json.gz"
response = requests.get(url)
with open("singpostcode.json.gz", "wb") as f:
    f.write(response.content)

with gzip.open("singpostcode.json.gz", "rt", encoding="utf-8") as f:
    data = json.load(f)

postal_df = pd.DataFrame(data)
print(postal_df.head())


                                             ADDRESS BLK_NO  \
0  1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...      1   
1  11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...    11A   
2           5A MARINA GARDENS DRIVE SINGAPORE 018910     5A   
3  2 CENTRAL BOULEVARD CENTRAL BOULEVARD TOWERS S...      2   
4  21 PARK STREET MARINA BAY MRT STATION SINGAPOR...     21   

                            BUILDING     LATITUDE    LONGITUDE   LONGTITUDE  \
0  SINGAPORE CHINESE CULTURAL CENTRE  1.275804635   103.849615   103.849615   
1              TEMPORARY SITE OFFICE  1.274949694  103.8516652  103.8516652   
2                                NIL  1.279586788  103.8689557  103.8689557   
3           CENTRAL BOULEVARD TOWERS   1.27974389  103.8515913  103.8515913   
4      MARINA BAY MRT STATION (NS27)  1.276409263  103.8545953  103.8545953   

   POSTAL             ROAD_NAME                                 SEARCHVAL  \
0  018906     STRAITS BOULEVARD         SINGAPORE CHINESE CULTURAL CE

In [39]:
acra_df=acra_df[['entity_name','reg_postal_code','entity_name']]

In [38]:
acra_df

,uen,issuance_agency_desc,uen_status_desc,entity_name,entity_type_desc,uen_issue_date,reg_street_name,reg_postal_code
0,201201936C,ACRA,Deregistered,MILLENNIUM AUTOMATION & SYSTEMS PTE. LTD.,Local Company,2012-01-26,BENCOOLEN STREET,189648
1,53250767C,ACRA,Deregistered,PARTY ART ENTERTAINMENT,Sole Proprietorship/ Partnership,2013-12-13,CHOA CHU KANG AVENUE 4,680448
2,199203392Z,ACRA,Deregistered,DE COMFORT TRADING & DEVELOPMENT PTE LTD,Local Company,1992-06-30,SEGAR ROAD,670481
3,201305833D,ACRA,Deregistered,EXCEL CAPITAL PTE. LTD.,Local Company,2013-03-06,ANSON ROAD,079903
4,199600940G,ACRA,Registered,DYNASOURCE ENTERPRISE PTE LTD,Local Company,1996-02-06,PASIR RIS GROVE,518217
...,...,...,...,...,...,...,...,...
1668057,201943451K,ACRA,Deregistered,ASIAN METALLURGY PTE. LTD.,Local Company,2019-12-25,PAYA LEBAR ROAD,409051
1668058,201414664H,ACRA,Deregistered,TRICAB (SINGAPORE) PTE. LTD.,Local Company,2014-05-21,VENTURE DRIVE,608526
1668059,200503172K,ACRA,Deregistered,WSL PTE. LTD.,Local Company,2005-03-10,JALAN BERJAYA,578595
1668060,195100108G,ACRA,Deregistered,LAM SIONG COMPANY LIMITED,Local Company,1951-09-25,BEACH ROAD,189686


In [40]:
acra_df['reg_postal_code'] = acra_df['reg_postal_code'].astype(str)
postal_df['POSTAL'] = postal_df['POSTAL'].astype(str)
merged = pd.merge(
    acra_df,
    postal_df,
    left_on='reg_postal_code',
    right_on='POSTAL',
    how='left'
)

In [41]:
acra_df=merged[['entity_name','ADDRESS','LATITUDE','LONGITUDE']]

In [25]:
mrt=pd.read_csv('https://raw.githubusercontent.com/elliotwutingfeng/singapore_train_station_coordinates/refs/heads/main/all_stations.csv')

In [26]:
mrt

,station_code,station_name,lat,lon,source,comment
0,BP1,Choa Chu Kang,1.384755,103.744538,onemap,NaN
1,BP2,South View,1.380298,103.745292,onemap,NaN
2,BP3,Keat Hong,1.378603,103.749056,onemap,NaN
3,BP4,Teck Whye,1.376685,103.753712,onemap,NaN
4,BP5,Phoenix,1.378615,103.757996,onemap,NaN
...,...,...,...,...,...,...
272,TE28,Siglap,1.310009,103.930026,onemap,NaN
273,TE29,Bayshore,1.313122,103.942310,onemap,NaN
274,TE30,Bedok South,1.316684,103.949311,onemap,tel_5
275,TE31,Sungei Bedok,1.320401,103.957184,onemap,tel_5


In [27]:
mrt_df = mrt[['station_code','station_name','lat','lon']]
print(mrt_df.head())

  station_code   station_name       lat         lon
0          BP1  Choa Chu Kang  1.384755  103.744538
1          BP2     South View  1.380298  103.745292
2          BP3      Keat Hong  1.378603  103.749056
3          BP4      Teck Whye  1.376685  103.753712
4          BP5        Phoenix  1.378615  103.757996


In [28]:
bus_stops = []
skip = 0

while True:
    url = f"https://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip={skip}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    batch = resp.json()['value']
    if not batch:
        break
    bus_stops.extend(batch)
    skip += len(batch)

bus_stops_df = pd.DataFrame(bus_stops)
print(bus_stops_df.head())

  BusStopCode       RoadName          Description  Latitude   Longitude
0       01012    Victoria St  Hotel Grand Pacific  1.296848  103.852536
1       01013    Victoria St      St. Joseph's Ch  1.297710  103.853225
2       01019    Victoria St      Bras Basah Cplx  1.296990  103.853022
3       01029  Nth Bridge Rd         Opp Natl Lib  1.296673  103.854414
4       01039  Nth Bridge Rd           Bugis Cube  1.298208  103.855491


In [51]:
acra_df_lat_lon=acra_df[['LATITUDE','LONGITUDE','entity_name']]

In [52]:
acra_df_lat_lon['LATITUDE'] = pd.to_numeric(acra_df_lat_lon['LATITUDE'], errors='coerce')
acra_df_lat_lon['LONGITUDE'] = pd.to_numeric(acra_df_lat_lon['LONGITUDE'], errors='coerce')

In [53]:
import numpy as np
from sklearn.neighbors import BallTree
acra_latlon = np.deg2rad(acra_df_lat_lon[['LATITUDE', 'LONGITUDE']].values)
bus_latlon = np.deg2rad(bus_stops_df[['Latitude', 'Longitude']].values)
mrt_latlon = np.deg2rad(mrt_df[['lat', 'lon']].values)

In [62]:

acra_latlon = np.deg2rad(acra_df_lat_lon[['LATITUDE', 'LONGITUDE']].values)
bus_latlon = np.deg2rad(bus_stops_df[['Latitude', 'Longitude']].values)
mrt_latlon = np.deg2rad(mrt_df[['lat', 'lon']].values)

bus_tree = BallTree(bus_latlon, metric='haversine')
mrt_tree = BallTree(mrt_latlon, metric='haversine')

dist_bus, idx_bus = bus_tree.query(acra_latlon, k=5)
dist_mrt, idx_mrt = mrt_tree.query(acra_latlon, k=5)

dist_bus_m = dist_bus[:, 0] * 6371000
dist_mrt_m = dist_mrt[:, 0] * 6371000

acra_df_lat_lon['nearest_bus_stop'] = bus_stops_df.iloc[idx_bus[:, 0]].reset_index(drop=True)['BusStopCode']
acra_df_lat_lon['bus_distance_m'] = dist_bus_m
acra_df_lat_lon['nearest_mrt'] = mrt_df.iloc[idx_mrt[:, 0]].reset_index(drop=True)['station_code']
acra_df_lat_lon['mrt_distance_m'] = dist_mrt_m

print(acra_df_lat_lon.head())
all_bus_indices = np.unique(idx_bus.flatten())
all_mrt_indices = np.unique(idx_mrt.flatten())
unique_bus_stops = bus_stops_df.iloc[all_bus_indices].copy()
unique_mrt_stations = mrt_df.iloc[all_mrt_indices].copy()
unique_bus_stops = unique_bus_stops.rename(
    columns={
        'BusStopCode': 'stop_id',
        'Description': 'stop_name',
        'Latitude': 'lat',
        'Longitude': 'lon'
    }
)
unique_bus_stops['type'] = 'bus_stop'

unique_mrt_stations = unique_mrt_stations.rename(
    columns={
        'station_name': 'stop_name',
        'lat': 'lat',
        'lon': 'lon'
    }
)
unique_mrt_stations['stop_id'] = unique_mrt_stations.get('station_code', unique_mrt_stations.index)
unique_mrt_stations['type'] = 'mrt_station'

cols = ['stop_id', 'stop_name', 'lat', 'lon', 'type']

all_stops_df = pd.concat([
    unique_bus_stops[cols],
    unique_mrt_stations[cols]
], ignore_index=True)



   LATITUDE   LONGITUDE                                entity_name  \
0  1.301632  103.853343  MILLENNIUM AUTOMATION & SYSTEMS PTE. LTD.   
1  1.380223  103.737347                    PARTY ART ENTERTAINMENT   
2  1.380138  103.737337                    PARTY ART ENTERTAINMENT   
3  1.389456  103.772226   DE COMFORT TRADING & DEVELOPMENT PTE LTD   
4  1.276053  103.846115                    EXCEL CAPITAL PTE. LTD.   

                                 entity_name nearest_bus_stop  bus_distance_m  \
0  MILLENNIUM AUTOMATION & SYSTEMS PTE. LTD.            07517       98.199417   
1                    PARTY ART ENTERTAINMENT            44621      113.830494   
2                    PARTY ART ENTERTAINMENT            44621      104.460789   
3   DE COMFORT TRADING & DEVELOPMENT PTE LTD            44691      157.432570   
4                    EXCEL CAPITAL PTE. LTD.            03223       45.261823   

  nearest_mrt  mrt_distance_m  
0        DT13      254.929158  
1         JS2      316.71057

In [68]:
all_stops_df

,stop_id,stop_name,lat,lon,type
0,01012,Hotel Grand Pacific,1.296848,103.852536,bus_stop
1,01013,St. Joseph's Ch,1.297710,103.853225,bus_stop
2,01019,Bras Basah Cplx,1.296990,103.853022,bus_stop
3,01029,Opp Natl Lib,1.296673,103.854414,bus_stop
4,01039,Bugis Cube,1.298208,103.855491,bus_stop
...,...,...,...,...,...
5407,TE28,Siglap,1.310009,103.930026,mrt_station
5408,TE29,Bayshore,1.313122,103.942310,mrt_station
5409,TE30,Bedok South,1.316684,103.949311,mrt_station
5410,TE31,Sungei Bedok,1.320401,103.957184,mrt_station


In [64]:
od_bus_df['ORIGIN_PT_CODE'] = od_bus_df['ORIGIN_PT_CODE'].astype(str)
od_bus_df['DESTINATION_PT_CODE'] = od_bus_df['DESTINATION_PT_CODE'].astype(str)
all_stops_df['stop_id'] = all_stops_df['stop_id'].astype(str)

In [69]:
valid_mrt_stations = set(all_stops_df.query('type == "mrt_station"')['stop_id'])

filtered_od_train = od_train_df[
    od_train_df['ORIGIN_PT_CODE'].isin(valid_mrt_stations) &
    od_train_df['DESTINATION_PT_CODE'].isin(valid_mrt_stations)
].copy()


In [70]:
valid_bus_stops = set(all_stops_df.query('type == "bus_stop"')['stop_id'])

filtered_od_bus = od_bus_df[
    od_bus_df['ORIGIN_PT_CODE'].isin(valid_bus_stops) &
    od_bus_df['DESTINATION_PT_CODE'].isin(valid_bus_stops)
].copy()


In [102]:
# Filter for weekday AM peak (commute-to-work)
am_peak_hours = [7,8,9]
pm_peak_hours = [18,19,20]

filtered_od_train = od_train_df[
    (od_train_df['DAY_TYPE'] == 'WEEKDAY') &
    (od_train_df['TIME_PER_HOUR'].isin(am_peak_hours))
]

filtered_od_bus = od_bus_df[
    (od_bus_df['DAY_TYPE'] == 'WEEKDAY') &
    (od_bus_df['TIME_PER_HOUR'].isin(am_peak_hours))
]



In [103]:
top_od_bus = (
    filtered_od_bus.groupby(['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'])['TOTAL_TRIPS']
    .sum()
    .reset_index()
    .sort_values('TOTAL_TRIPS', ascending=False)
)

top_od_train = (
    filtered_od_train.groupby(['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'])['TOTAL_TRIPS']
    .sum()
    .reset_index()
    .sort_values('TOTAL_TRIPS', ascending=False)
)

In [106]:
top_od=pd.concat([top_od_bus,top_od_train])

In [107]:
top_od.head()

,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
87820,46219,46109,67802
87307,46101,46211,67145
87379,46109,45139,52534
87096,46069,46009,29328
91248,47009,46101,26742


In [76]:
threshold = top_od['TOTAL_TRIPS'].quantile(0.75)
top_od = top_od[top_od['TOTAL_TRIPS'] > threshold]

In [109]:
all_stops_df['stop_id'] = all_stops_df['stop_id'].astype(str)
top_od['ORIGIN_PT_CODE'] = top_od['ORIGIN_PT_CODE'].astype(str)
top_od['DESTINATION_PT_CODE'] = top_od['DESTINATION_PT_CODE'].astype(str)

top_od = top_od.merge(
    all_stops_df[['stop_id', 'lat', 'lon']],
    left_on='ORIGIN_PT_CODE',
    right_on='stop_id',
    how='left'
).rename(columns={'lat': 'origin_lat', 'lon': 'origin_lon'})

top_od = top_od.merge(
    all_stops_df[['stop_id', 'lat', 'lon']],
    left_on='DESTINATION_PT_CODE',
    right_on='stop_id',
    how='left'
).rename(columns={'lat': 'dest_lat', 'lon': 'dest_lon'})

top_od = top_od.drop(columns=['stop_id_x', 'stop_id_y'])


In [112]:
top_od=top_od.dropna()

In [113]:
top_od.head(10)

,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,origin_lat,origin_lon,dest_lat,dest_lon
2,46109,45139,52534,1.446941,103.769253,1.425260,103.761781
3,46069,46009,29328,1.442889,103.769338,1.436946,103.785936
4,47009,46101,26742,1.437553,103.786282,1.444608,103.767749
5,53389,53231,24101,1.358045,103.845169,1.350900,103.847980
6,45139,46101,23110,1.425260,103.761781,1.444608,103.767749
7,63059,63039,21721,1.364377,103.889841,1.360260,103.885384
8,22599,22009,20018,1.344361,103.703451,1.339323,103.705457
9,59181,59072,19824,1.434166,103.842188,1.429383,103.835493
10,45421,45329,17557,1.402749,103.750509,1.397438,103.747865
11,53281,53009,16685,1.346656,103.854321,1.350455,103.849881


In [114]:
print(top_od.columns)

Index(['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE', 'TOTAL_TRIPS', 'origin_lat',
       'origin_lon', 'dest_lat', 'dest_lon'],
      dtype='object')


In [115]:
import geopandas as gpd
from shapely.geometry import LineString

top_od['od_line'] = top_od.apply(
    lambda row: LineString([(row['origin_lon'], row['origin_lat']), (row['dest_lon'], row['dest_lat'])]),
    axis=1
)

od_gdf = gpd.GeoDataFrame(top_od, geometry='od_line', crs='EPSG:4326')


In [116]:
od_gdf = od_gdf.to_crs(3414)
bike_gdf = bike_gdf.to_crs(3414)
road_gdf = road_gdf.to_crs(3414)


In [117]:
def check_od_coverage_with_index(od_gdf, bike_gdf):
    bike_gdf = bike_gdf.explode(index_parts=False, ignore_index=True)
    bike_gdf = bike_gdf[~bike_gdf.geometry.is_empty]
    bike_gdf = bike_gdf[bike_gdf.geometry.type == 'LineString']
    bike_sindex = bike_gdf.sindex

    def is_covered(line):
        possible = list(bike_sindex.intersection(line.bounds))
        if not possible:
            return False
        return bike_gdf.iloc[possible].geometry.intersects(line).any()

    return od_gdf.geometry.apply(is_covered)


od_gdf['bike_covered'] = check_od_coverage_with_index(od_gdf, bike_gdf)


In [118]:
summary = (
    od_gdf['bike_covered']
    .value_counts(normalize=True)
    .rename_axis('bike_covered')
    .reset_index(name='percentage')
)
summary['percentage'] = summary['percentage'] * 100


In [119]:
summary

,bike_covered,percentage
0,True,95.138666
1,False,4.861334


In [120]:
uncovered_od = od_gdf[~od_gdf['bike_covered']].copy()


In [121]:
uncovered_od['buffer'] = uncovered_od.geometry.buffer(40)


In [122]:
road_gdf = road_gdf.explode(index_parts=False, ignore_index=True)
road_gdf = road_gdf[~road_gdf.geometry.is_empty]
road_gdf = road_gdf[road_gdf.geometry.type == 'LineString']


In [123]:
buffer_gdf = gpd.GeoDataFrame(uncovered_od[['buffer']], geometry='buffer', crs=road_gdf.crs)
road_to_buffer = gpd.sjoin(
    road_gdf, buffer_gdf, how='inner', predicate='intersects'
).drop_duplicates(subset=road_gdf.columns.tolist())


In [124]:
bike_gdf = bike_gdf.explode(index_parts=False, ignore_index=True)
bike_gdf = bike_gdf[~bike_gdf.geometry.is_empty]
bike_gdf = bike_gdf[bike_gdf.geometry.type == 'LineString']

road_to_buffer['has_bike'] = road_to_buffer.geometry.apply(lambda geom: bike_gdf.geometry.intersects(geom).any())

proposed_roads = road_to_buffer[~road_to_buffer['has_bike']].copy()


In [125]:
proposed_roads['length_m'] = proposed_roads.geometry.length
proposed_roads = proposed_roads.sort_values('length_m', ascending=False)
proposed_roads['cum_length_km'] = proposed_roads['length_m'].cumsum() / 1000

final_recommend = proposed_roads[proposed_roads['cum_length_km'] <= 100]


In [126]:
import folium

final_recommend_wgs84 = final_recommend.to_crs(4326)

center_lat = final_recommend_wgs84.geometry.centroid.y.mean()
center_lon = final_recommend_wgs84.geometry.centroid.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="cartodbpositron")

for line in final_recommend_wgs84.geometry:
    if line.geom_type == 'LineString':
        coords = [(lat, lon) for lon, lat, *_ in line.coords]
        folium.PolyLine(coords, color='orange', weight=5, opacity=0.8, tooltip="Proposed Cycling Path").add_to(m)
    elif line.geom_type == 'MultiLineString':
        for seg in line:
            coords = [(lat, lon) for lon, lat, *_ in seg.coords]
            folium.PolyLine(coords, color='orange', weight=5, opacity=0.8, tooltip="Proposed Cycling Path").add_to(m)


m


In [127]:
m.save('map.png')